In [1]:
#Data saved in csv/excel
import glob, os
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [23]:
reg = pd.read_csv('C:/Users/tsb7592/Downloads/registration_data_distinct.csv',index_col=0)

In [24]:
reg.shape

(3260, 13)

In [34]:
reg.columns =['PIN', 'RegistrationID', 'DeviceID', 'Age','Education', 'MothersEducation',
       'FathersEducation', 'GuardiansEducation','ExaminerName', 'Gender', 'Handedness',
       'Race','Ethnicity']

In [36]:
#reg.to_csv('C:/Users/tsb7592/Downloads/registration_data_distinct.csv')

In [202]:
orn = pd.read_csv('C:/Users/tsb7592/Downloads/OR_norm.csv', index_col=0)

In [203]:
orn =orn.drop_duplicates()
orn.shape

(1081858, 11)

In [204]:
#Importing the FormWords
xls =  pd.read_excel('C:/Users/tsb7592/Box/Norming Psychometrics/Data Dictionaries/R/OR/OR Forms for Norming 07.08.2020_WithWords.xlsx',sheet_name = "Formatted_Forms")

In [233]:
#routing forms are everything that contain “Routing_” in the column Form in FormWords. Remember that routing form item IDs overlap with Live Form item IDs so you can’t just do a match across all items taken.
xls=xls[xls['Form'].str.contains("Routing")]

In [234]:
#Change the column tpye from obeject to numeric
#Select the 3 needed key values
data =orn[(orn.Key == 'Score')| (orn.Key =='Response') | (orn.Key == 'Position')]
data['Value'] =pd.to_numeric(data['Value'])

In [235]:
data

,PIN,RegistrationID,DeviceID,ResponseID,ItemID,AssessmentName,InstrumentTitle,TestName,InstrumentID,Key,Value
3168,MAP10001,C2D64D16-D416-4676-A23E-C200CCC15F21,49CF3CA6-E25F-4420-8E33-7099FFA052DE,4BB5F2C8-B87A-44DF-9F3C-39690D067DD6,OR_Title,Assessment 1,NIH Toolbox Oral Reading Recognition Instructi...,OR_norm,EF1F7A5F-2844-446B-AA8C-5B69569D35AC,Response,1
3170,MAP10001,C2D64D16-D416-4676-A23E-C200CCC15F21,49CF3CA6-E25F-4420-8E33-7099FFA052DE,4BB5F2C8-B87A-44DF-9F3C-39690D067DD6,OR_Title,Assessment 1,NIH Toolbox Oral Reading Recognition Instructi...,OR_norm,EF1F7A5F-2844-446B-AA8C-5B69569D35AC,Position,1
3178,MAP10001,C2D64D16-D416-4676-A23E-C200CCC15F21,49CF3CA6-E25F-4420-8E33-7099FFA052DE,A18FEED4-70FC-43B7-AF3E-DA2D5AEB2B7B,OR_Instruction,Assessment 1,NIH Toolbox Oral Reading Recognition Instructi...,OR_norm,EF1F7A5F-2844-446B-AA8C-5B69569D35AC,Response,1
3180,MAP10001,C2D64D16-D416-4676-A23E-C200CCC15F21,49CF3CA6-E25F-4420-8E33-7099FFA052DE,A18FEED4-70FC-43B7-AF3E-DA2D5AEB2B7B,OR_Instruction,Assessment 1,NIH Toolbox Oral Reading Recognition Instructi...,OR_norm,EF1F7A5F-2844-446B-AA8C-5B69569D35AC,Position,2
3188,MAP10001,C2D64D16-D416-4676-A23E-C200CCC15F21,49CF3CA6-E25F-4420-8E33-7099FFA052DE,CE2D4F9D-DC9C-4701-9564-38D5F11550FA,LARE135,Assessment 1,NIH Toolbox Oral Reading Recognition Test Norm...,OR_norm,3EF3780D-5464-430F-BD86-BDAFBC7A0F0F,Response,1
...,...,...,...,...,...,...,...,...,...,...,...
24812011,MSL15016,496005C2-9734-4595-9EE0-D621E4F39FD6,DDC1B727-13A3-4C52-8E5F-AD9E80B28A37,15167AFD-D3C4-4070-8F6C-6348858A56A3,LARE165,Assessment 1,NIH Toolbox Oral Reading Recognition Test Norm...,OR_norm,36015383-BA69-4249-A1D4-03BE94303952,Score,1
24812013,MSL15016,496005C2-9734-4595-9EE0-D621E4F39FD6,DDC1B727-13A3-4C52-8E5F-AD9E80B28A37,15167AFD-D3C4-4070-8F6C-6348858A56A3,LARE165,Assessment 1,NIH Toolbox Oral Reading Recognition Test Norm...,OR_norm,36015383-BA69-4249-A1D4-03BE94303952,Position,24
24812022,MSL15016,496005C2-9734-4595-9EE0-D621E4F39FD6,DDC1B727-13A3-4C52-8E5F-AD9E80B28A37,6547E12E-B43C-472D-BEA4-7C06090F9C50,LARE254,Assessment 1,NIH Toolbox Oral Reading Recognition Test Norm...,OR_norm,36015383-BA69-4249-A1D4-03BE94303952,Response,1
24812023,MSL15016,496005C2-9734-4595-9EE0-D621E4F39FD6,DDC1B727-13A3-4C52-8E5F-AD9E80B28A37,6547E12E-B43C-472D-BEA4-7C06090F9C50,LARE254,Assessment 1,NIH Toolbox Oral Reading Recognition Test Norm...,OR_norm,36015383-BA69-4249-A1D4-03BE94303952,Score,1


In [236]:
data = data.pivot_table(index=['RegistrationID','PIN','ItemID','DeviceID'], columns='Key', values='Value').reset_index()

In [237]:
data=data.drop_duplicates()

In [238]:
data.head()

Key,RegistrationID,PIN,ItemID,DeviceID,Position,Response,Score
0,000AB98A-683B-4BAF-B95A-50F7DB253184,SDA40009,LARE070,2A74DD06-DE18-4CF3-8568-86E6822D8A06,18.0,1.0,1.0
1,000AB98A-683B-4BAF-B95A-50F7DB253184,SDA40009,LARE080,2A74DD06-DE18-4CF3-8568-86E6822D8A06,6.0,1.0,1.0
2,000AB98A-683B-4BAF-B95A-50F7DB253184,SDA40009,LARE089,2A74DD06-DE18-4CF3-8568-86E6822D8A06,13.0,2.0,0.0
3,000AB98A-683B-4BAF-B95A-50F7DB253184,SDA40009,LARE090,2A74DD06-DE18-4CF3-8568-86E6822D8A06,12.0,1.0,1.0
4,000AB98A-683B-4BAF-B95A-50F7DB253184,SDA40009,LARE092,2A74DD06-DE18-4CF3-8568-86E6822D8A06,4.0,1.0,1.0


In [239]:
data =data[data['ItemID'].str.contains("LARE")]

In [240]:
data.shape

(84459, 7)

In [241]:
rv =pd.merge(data, xls, on='ItemID')

In [242]:
data.Position.describe()


count    84459.000000
mean        12.917368
std          7.189742
min          1.000000
25%          7.000000
50%         13.000000
75%         19.000000
max         25.000000
Name: Position, dtype: float64

In [243]:
rv =rv.drop_duplicates()
rv.shape

(68822, 9)

In [244]:
rv.head()

,RegistrationID,PIN,ItemID,DeviceID,Position,Response,Score,Form,Vocabulary
0,000AB98A-683B-4BAF-B95A-50F7DB253184,SDA40009,LARE080,2A74DD06-DE18-4CF3-8568-86E6822D8A06,6.0,1.0,1.0,Routing_10-SC,meant
1,0019836B-DF41-4AD6-9911-03A553B5CD7F,SBM40006,LARE080,6771869B-34B5-4934-8295-6F2F454C47FF,5.0,1.0,1.0,Routing_10-SC,meant
2,0031B79A-BBE0-4749-96C9-0AA8D70C1EE6,SNA30004,LARE080,09E93F6C-C040-4946-8683-96A3DF1440CA,7.0,1.0,1.0,Routing_10-SC,meant
3,0033BCF5-7F7F-4441-A61B-E164924FF3C4,WLA17028,LARE080,92883D82-9973-4077-901D-A68BC204CE78,4.0,1.0,1.0,Routing_10-SC,meant
4,00EEEFB6-968E-4308-9F42-EAD562090296,WLA10014,LARE080,341340EF-2410-444A-95F0-4709A5CF34D2,6.0,1.0,1.0,Routing_10-SC,meant


## Filter by different age

In [245]:
four = reg[reg.Age<=4]

In [246]:
df4 =pd.merge(rv, four, on=['RegistrationID', 'PIN','DeviceID'])

In [247]:
df4.drop_duplicates()
df4.shape

(1075, 19)

In [248]:
df4 =df4[(df4.Position >=3) & (df4.Position <=7)]

In [250]:
df4

,RegistrationID,PIN,ItemID,DeviceID,Position,Response,Score,Form,Vocabulary,Age,Education,MothersEducation,FathersEducation,GuardiansEducation,ExaminerName,Gender,Handedness,Race,Ethnicity
1,9754308E-64C0-42A6-AA35-2DD5AA424212,MSL11011,LARE092,DA90E71A-E0DF-43AA-8D44-E13ACC67B72A,3.0,1.0,1.0,Routing_10-SC,abate,3.0,8.0,21.0,NaN,NaN,Donneice M,2.0,1.0,1,1.0
4,9754308E-64C0-42A6-AA35-2DD5AA424212,MSL11011,LARE143,DA90E71A-E0DF-43AA-8D44-E13ACC67B72A,7.0,1.0,1.0,Routing_7-9,amplify,3.0,8.0,21.0,NaN,NaN,Donneice M,2.0,1.0,1,1.0
5,9754308E-64C0-42A6-AA35-2DD5AA424212,MSL11011,LARE143,DA90E71A-E0DF-43AA-8D44-E13ACC67B72A,7.0,1.0,1.0,Routing_10-SC,amplify,3.0,8.0,21.0,NaN,NaN,Donneice M,2.0,1.0,1,1.0
8,9754308E-64C0-42A6-AA35-2DD5AA424212,MSL11011,LARE172,DA90E71A-E0DF-43AA-8D44-E13ACC67B72A,6.0,1.0,1.0,Routing_7-9,conscious,3.0,8.0,21.0,NaN,NaN,Donneice M,2.0,1.0,1,1.0
9,9754308E-64C0-42A6-AA35-2DD5AA424212,MSL11011,LARE172,DA90E71A-E0DF-43AA-8D44-E13ACC67B72A,6.0,1.0,1.0,Routing_10-SC,conscious,3.0,8.0,21.0,NaN,NaN,Donneice M,2.0,1.0,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1066,FE419795-2022-4FE0-AA40-A26B7DE198A7,MAP04005,LARE028,570A856B-E5EC-4DBD-B4EE-B10C51BC919A,4.0,2.0,0.0,Routing_7-9,did,4.0,2.0,16.0,NaN,NaN,Clare S,2.0,1.0,0,NaN
1069,FE419795-2022-4FE0-AA40-A26B7DE198A7,MAP04005,LARE008,570A856B-E5EC-4DBD-B4EE-B10C51BC919A,5.0,2.0,0.0,Routing_4,N,4.0,2.0,16.0,NaN,NaN,Clare S,2.0,1.0,0,NaN
1070,FE419795-2022-4FE0-AA40-A26B7DE198A7,MAP04005,LARE008,570A856B-E5EC-4DBD-B4EE-B10C51BC919A,5.0,2.0,0.0,Routing_5-6,N,4.0,2.0,16.0,NaN,NaN,Clare S,2.0,1.0,0,NaN
1071,FE419795-2022-4FE0-AA40-A26B7DE198A7,MAP04005,LARE021,570A856B-E5EC-4DBD-B4EE-B10C51BC919A,3.0,2.0,0.0,Routing_4,on,4.0,2.0,16.0,NaN,NaN,Clare S,2.0,1.0,0,NaN


In [260]:
df =pd.merge(rv, reg, on=['RegistrationID', 'PIN','DeviceID'])

In [191]:
df.RegistrationID.nunique()

3412

In [192]:
reg.RegistrationID.nunique()

3260

In [261]:
dd =df.groupby(['Form','ItemID','Vocabulary'])['Score'].mean().reset_index()
dd

,Form,ItemID,Vocabulary,Score
0,Routing_10-SC,LARE080,meant,0.961071
1,Routing_10-SC,LARE092,abate,0.870267
2,Routing_10-SC,LARE135,viscous,0.261269
3,Routing_10-SC,LARE143,amplify,0.837920
4,Routing_10-SC,LARE144,bucolic,0.483723
5,Routing_10-SC,LARE172,conscious,0.770290
6,Routing_10-SC,LARE175,aesthetic,0.768562
7,Routing_10-SC,LARE198,nonchalant,0.785892
8,Routing_10-SC,LARE251,recrudescence,0.305219
9,Routing_10-SC,LARE264,reconnaissance,0.525459


In [262]:
ct=df.groupby(['Form','ItemID','Vocabulary'])['Position'].count().reset_index()
ct

,Form,ItemID,Vocabulary,Position
0,Routing_10-SC,LARE080,meant,1644
1,Routing_10-SC,LARE092,abate,1611
2,Routing_10-SC,LARE135,viscous,2396
3,Routing_10-SC,LARE143,amplify,1962
4,Routing_10-SC,LARE144,bucolic,2396
5,Routing_10-SC,LARE172,conscious,2760
6,Routing_10-SC,LARE175,aesthetic,2761
7,Routing_10-SC,LARE198,nonchalant,2410
8,Routing_10-SC,LARE251,recrudescence,2395
9,Routing_10-SC,LARE264,reconnaissance,2396


In [253]:
st=df4.groupby(['Form','ItemID','Vocabulary'])['Age'].std().reset_index()
st

,Form,ItemID,Vocabulary,Age
0,Routing_10-SC,LARE092,abate,NaN
1,Routing_10-SC,LARE143,amplify,NaN
2,Routing_10-SC,LARE172,conscious,NaN
3,Routing_10-SC,LARE175,aesthetic,NaN
4,Routing_10-SC,LARE251,recrudescence,NaN
5,Routing_4,LARE005,C,0.000000
6,Routing_4,LARE008,N,0.131306
7,Routing_4,LARE021,on,0.128037
8,Routing_4,LARE028,did,0.132453
9,Routing_4,LARE034,box,0.000000


In [263]:
age =df.groupby(['Form','ItemID','Vocabulary'])['Age'].mean().reset_index()
age

,Form,ItemID,Vocabulary,Age
0,Routing_10-SC,LARE080,meant,22.833130
1,Routing_10-SC,LARE092,abate,23.150404
2,Routing_10-SC,LARE135,viscous,31.422482
3,Routing_10-SC,LARE143,amplify,20.472690
4,Routing_10-SC,LARE144,bucolic,31.422482
5,Routing_10-SC,LARE172,conscious,28.347968
6,Routing_10-SC,LARE175,aesthetic,28.341676
7,Routing_10-SC,LARE198,nonchalant,31.274200
8,Routing_10-SC,LARE251,recrudescence,31.431020
9,Routing_10-SC,LARE264,reconnaissance,31.422482


In [255]:
result =pd.merge(dd, ct, on=['Form','ItemID','Vocabulary'])
result =pd.merge(result, age, on=['Form','ItemID','Vocabulary'])
result =pd.merge(result, st, on=['Form','ItemID','Vocabulary'])

In [256]:
result.columns=['Form', 'ItemID', 'Vocabulary', 'Proportion_Correct', 'N','MeanAge','SDAge']

In [257]:
result

,Form,ItemID,Vocabulary,Proportion_Correct,N,MeanAge,SDAge
0,Routing_10-SC,LARE092,abate,1.000000,1,3.000000,NaN
1,Routing_10-SC,LARE143,amplify,1.000000,1,3.000000,NaN
2,Routing_10-SC,LARE172,conscious,1.000000,1,3.000000,NaN
3,Routing_10-SC,LARE175,aesthetic,1.000000,1,3.000000,NaN
4,Routing_10-SC,LARE251,recrudescence,1.000000,1,3.000000,NaN
5,Routing_4,LARE005,C,0.568627,51,4.000000,0.000000
6,Routing_4,LARE008,N,0.517241,58,3.982759,0.131306
7,Routing_4,LARE021,on,0.196721,61,3.983607,0.128037
8,Routing_4,LARE028,did,0.087719,57,3.982456,0.132453
9,Routing_4,LARE034,box,0.098039,51,4.000000,0.000000


In [259]:
result.to_csv('C:/Users/tsb7592/Downloads/OR_routingform_below4.csv', index=False)